In [7]:
import copy 
# The eight movement directions possible for a chess queen
RAYS = [(1, 0), (1, -1), (0, -1), (-1, -1),
        (-1, 0), (-1, 1), (0, 1), (1, 1)]


In [11]:
class GameState:

    def __init__(self, xlen = 3, ylen=2):
        """The GameState class constructor performs required
        initializations when an instance is created. The class
        should:
        
        1) Keep track of which cells are open/closed
        2) Identify which player has initiative
        3) Record the current location of each player
        
        Parameters
        ----------
        self:
            instance methods automatically take "self" as an
            argument in python
        
        Returns
        -------
        None
        """
        self._xlen = xlen
        self._ylen = ylen
        self._board = ([[0] * self._ylen for x in range(self._xlen)])
        self._board[2][1] = 1
        self._player = 0 # id 0 for player 1, 1 for player 2
        self._locations = [None, None]
        
    @property
    def hashable(self):
        from itertools import chain
        return tuple(chain(*self._board)) + tuple(self._locations) + (self._player, )
    
        
    def actions(self):
        """ Return a list of legal actions for the active player 
        
        You are free to choose any convention to represent actions,
        but one option is to represent actions by the (row, column)
        of the endpoint for the token. For example, if your token is
        in (0, 0), and your opponent is in (1, 0) then the legal
        actions could be encoded as (0, 1) and (0, 2).
        """
        return self.liberties(self._locations[self._player])
    
    def player(self):
        """ Return the id of the active player 
        
        Hint: return 0 for the first player, and 1 for the second player
        """
        return self._player
    
    def result(self, action):
        """ Return a new state that results from applying the given
        action in the current state
        
        Hint: Check out the deepcopy module--do NOT modify the
        objects internal state in place
        """
        assert action in self.actions()
        resultState = copy.deepcopy(self)
        resultState._board[action[0]][action[1]] = 1
        resultState._locations[self._player] = action
        resultState._player ^= 1 
        return resultState
    
    def terminal_test(self):
        """ return True if the current state is terminal,
        and False otherwise
        
        Hint: an Isolation state is terminal if _either_
        player has no remaining liberties (even if the
        player is not active in the current state)
        """
        global call_counter
        call_counter += 1
        return not any(self.liberties(self._locations[0])) or not any(self.liberties(self._locations[1]))
    
    def liberties(self, loc):
        """ Return a list of all open cells in the
        neighborhood of the specified location.  The list 
        should include all open spaces in a straight line
        along any row, column or diagonal from the current
        position. (Tokens CANNOT move through obstacles
        or blocked squares in queens Isolation.)
        
        Note: if loc is None, then return all empty cells
        on the board
        """
        if loc is None:
            return [(x,y) for y in range(self._ylen) for x in range(self._xlen) if not self._board[x][y]]
        moves = []
        for dx, dy in RAYS:  # check each movement direction
            _x, _y = loc
            while 0 <= _x + dx < self._xlen and 0 <= _y + dy < self._ylen:
                _x, _y = _x + dx, _y + dy
                if self._board[_x][_y]:  # stop at any blocked cell
                    break
                moves.append((_x, _y))
        return moves
    
    def _has_liberties(self, player_id):
        """ Check to see if the specified player has any liberties """
        return any(self.liberties(self._locations[player_id]))
    
    def utility(self, player_id):
        """ return +inf if the game is terminal and the
        specified player wins, return -inf if the game
        is terminal and the specified player loses, and
        return 0 if the game is not terminal
        """
        if not self.terminal_test(): return 0
        player_id_is_active = (player_id == self.player())
        active_has_liberties = self._has_liberties(self.player())
        active_player_wins = (active_has_liberties == player_id_is_active)
        return float("inf") if active_player_wins else float("-inf")

In [16]:
# opening book implementation

import random

NUM_ROUNDS = 10

def build_table(num_rounds=NUM_ROUNDS):
    # You should run no more than `num_rounds` simulations -- the
    # goal of this quiz is to understand one possible way to develop
    # an opening book; not to develop a good one
    
    # NOTE: the GameState object is not hashable, and the python3
    #       runtime includes security features that make object
    #       hashes non-portable. There is a new attribute on
    #       GameState objects in this quiz called `hashable` that
    #       can be used as a dictionary key
    
    # TODO: return a table {k:v} where each k is a game state
    #       and each v is the best action to take in that state
    from collections import defaultdict, Counter
    book = defaultdict(Counter)
    for i in range(num_rounds):
        gameState = GameState()
        build_tree(gameState, book)
    return {k: max(v, key=v.get) for k, v in book.items()}

def build_tree(gameState, book, depth=2):
    if depth <= 0 or gameState.terminal_test():
        return -simulate(gameState)
    action = random.choice(gameState.actions())
    reward = build_tree(gameState.result(action), book, depth - 1)
    book[gameState.hashable][action] += reward
    return -reward

def simulate(gameState):
    player_id = gameState.player()
    while not gameState.terminal_test():
        gameState = gameState.result(random.choice(gameState.actions()))
    return -1 if gameState.utility(player_id) < 0 else 1

In [17]:
# test code 
call_counter = 0

book = build_table(10)

assert len(book) > 0, "Your opening book is empty"
assert all(isinstance(k, tuple) for k in book), \
    "All the keys should be `hashable`"
assert all(isinstance(v, tuple) and len(v) == 2 for v in book.values()), \
    "All the values should be tuples of (x, y) actions"
print("Looks like your book worked!")
print(book)

Looks like your book worked!
{(0, 0, 0, 0, 1, 1, (2, 0), None, 1): (1, 1), (0, 0, 0, 0, 0, 1, None, None, 0): (2, 0), (0, 0, 1, 0, 0, 1, (1, 0), None, 1): (0, 0), (0, 1, 0, 0, 0, 1, (0, 1), None, 1): (1, 0), (1, 0, 0, 0, 0, 1, (0, 0), None, 1): (0, 1)}
